<a href="https://colab.research.google.com/github/shtrausslearning/Deep-Learning-Projects/blob/main/fastai_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Предварительная обработка текста

#### Токенизация 

- Преобразование текста в список слов

#### Нумеризация 

- Составление списка всех уникальных слов словаря и преобразование каждого из них в число, соответствующее его индексу в словарке

In [25]:
import fastai
print(fastai.__version__)

2.7.10


In [2]:
# Импортируем данные из URLs данные
from fastai.text.all import untar_data, URLs

path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/imdb.vocab')]

In [5]:
from fastai.text.all import get_text_files

# Все текстовые документы
files = get_text_files(path,
                       folders=['train','test','unsup'])

print(len(files),'files')
print(files[0:2])
print(type(files)) # fastcode.foundation.L type

100000 files
[Path('/root/.fastai/data/imdb/test/pos/10136_10.txt'), Path('/root/.fastai/data/imdb/test/pos/4167_10.txt')]
<class 'fastcore.foundation.L'>


### Fastai Токенизация (WordTokeniser)

- Для вывода результатов `coll_repr(collection,n)`
- Токенизатор Английских слов в `fastai`, используется `SpaCy` (По умолчанию)
- Попробуем 

Примечание

- Токенизатор просто разделил слова и знаки препинания
- Разделила еще it's на it и 's (на самом деле раздельные слова)
- . oтделяется только когда оно часть окончания предложения, но не элементом акронима или числа

### Fastai Токенизация (Дополнительная функциональность)

Используя `Tokenizer` приносит в процесс токенизации дополнительную функциональность
- `xxbos` : начало текста
- `xxmaj` : слеующее слово начинается с большой буквой
- `xxunk` : слово неизвестно

In [26]:
# Откроем один пример

txt = files[0].open().read() # str
print(type(txt[:100]))
print(txt[:100])

# Токенизируем этот пример с WordTokenizer

from fastai.text.all import WordTokenizer,first,coll_repr

spacy_tkn = WordTokenizer()
tokens = first(spacy_tkn([txt]))

# tokens
print(coll_repr(tokens,30))

# Дополнительная функциональность

from fastai.text.all import Tokenizer

tkn = Tokenizer(spacy_tkn)
coll_repr(tkn(txt),30)

<class 'str'>
This is one of the best films I have ever seen! How anyone can knock this movie just befuddles my im
(#143) ['This','is','one','of','the','best','films','I','have','ever','seen','!','How','anyone','can','knock','this','movie','just','befuddles','my','imagination','!','First','of','all',',','Gooding',"'s",'and'...]


"(#156) ['xxbos','xxmaj','this','is','one','of','the','best','films','i','have','ever','seen','!','xxmaj','how','anyone','can','knock','this','movie','just','befuddles','my','imagination','!','xxmaj','first','of','all'...]"

### Fastai Токенизатор  (Токенизация подслов)

Выполняется в два этапа:
- Анализ корпуса документа в поиске наиболее часто встречающихся групп букв, из которых формулируется словарь
- Токенизация этого корпуса на основе словаря элементов подслов  
- Fastai list like object L()

In [22]:
# !pip install sentencepiece!=0.1.90,!=0.1.91

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00


In [30]:
from fastai.text.all import L 

txts = L(file.open().read() for file in files[:2000])
print(type(txts))

sample = txts[0]
print(sample)
print(type(sample))


<class 'fastcore.foundation.L'>
This is one of the best films I have ever seen! How anyone can knock this movie just befuddles my imagination! First of all, Gooding's and Harris's performances were simply spectacular, especially Gooding. That is the only way I can describe the acting: spectacular! You have to imagine how difficult it would be to play a character like that and pull it off; then you see Gooding, and his performance was magical. As for the plot, since it was based on a true person, it goes where the lives of the characters go. For all the action buffs, it might be a little slow, but then it's not an action film. I definitely give this movie a 10. It deserves nothing less!
<class 'str'>


In [42]:
from fastai.text.all import SubwordTokenizer

# Подловная токенизация, sz - размер словаря

def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  # sample return
  return ' | '.join(first(sp([txt]))[:40])


In [46]:
subword(200)

'▁ | T | h | i | s | ▁is | ▁on | e | ▁of | ▁the | ▁be | st | ▁film | s | ▁I | ▁h | a | ve | ▁ | e | ver | ▁ | se | en | ! | ▁ | H | ow | ▁ | an | y | on | e | ▁ca | n | ▁ | k | n | o | c'

- Используем словарь меньшего размера, каждый токен будет менльее число знаков
- Для представление предложениия требуется больше токенов

In [44]:
subword(1000)

'▁This | ▁is | ▁one | ▁of | ▁the | ▁best | ▁films | ▁I | ▁have | ▁ever | ▁seen | ! | ▁H | ow | ▁anyone | ▁can | ▁k | no | ck | ▁this | ▁movie | ▁just | ▁be | f | ud | d | le | s | ▁my | ▁ | im | ag | in | ation | ! | ▁F | ir | st | ▁of | ▁all'

- Используем более крупный словарь, наиболее распространенные слова окажутся в словаре

In [45]:
subword(10000)

"▁This | ▁is | ▁one | ▁of | ▁the | ▁best | ▁films | ▁I | ▁have | ▁ever | ▁seen | ! | ▁How | ▁anyone | ▁can | ▁knock | ▁this | ▁movie | ▁just | ▁be | fu | d | d | les | ▁my | ▁imagination | ! | ▁First | ▁of | ▁all | , | ▁Good | ing | ' | s | ▁and | ▁Harris | ' | s | ▁performances"

Выбор размера словаря подслов = компромисс 
- Более крупный размер озночает меньше число токенов в предложении, ускоренное обучение, меньше затарат для памяти 
-  Но с другой стороны, это озночает увеличение матриц вложении, что потребует изучения большего объема данных 

### Нумеризация с помощью Fastai

Разделив наши тексты на токены, нужно преобразовать их в числа

In [47]:
# sample
txt

"This is one of the best films I have ever seen! How anyone can knock this movie just befuddles my imagination! First of all, Gooding's and Harris's performances were simply spectacular, especially Gooding. That is the only way I can describe the acting: spectacular! You have to imagine how difficult it would be to play a character like that and pull it off; then you see Gooding, and his performance was magical. As for the plot, since it was based on a true person, it goes where the lives of the characters go. For all the action buffs, it might be a little slow, but then it's not an action film. I definitely give this movie a 10. It deserves nothing less!"

In [49]:
toks = tkn(txt)
toks

(#156) ['xxbos','xxmaj','this','is','one','of','the','best','films','i'...]

- Чтобы создать словарь, вызовем для `Numericalize` метод `setup`

In [52]:
# sample tokenising 
toks_sample = txts[:200].map(tkn)
toks_sample[0]

(#156) ['xxbos','xxmaj','this','is','one','of','the','best','films','i'...]

In [53]:
from fastai.text.all import Numericalize

num = Numericalize()
num.setup(toks_sample)

In [54]:
coll_repr(num.vocab,10)

"(#2104) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the'...]"

- Первые у нас специальные токены (**токены правил**), после них уже идут слова в порядке своей частности
- По умолчанию min_freq = 3 и max_vocab = 60000 слов, превышая эту сумму токены груперуются в **xxunk**

In [55]:
nums = num(toks)[:10]
nums

TensorText([  2,   8,  22,  16,  39,  14,   9, 112, 134,  19])

- декодируем данные обратно в string

In [56]:
' | '.join(num.vocab[i] for i in nums)

'xxbos | xxmaj | this | is | one | of | the | best | films | i'

### Создаем Батчи (Batches)

In [57]:
from fastai.text.all import LMDataLoader

nums_sample = toks_sample.map(num)

In [58]:
dl = LMDataLoader(nums_sample)

In [59]:
X,y = first(dl)
print(X.size(),y.size())

torch.Size([64, 72]) torch.Size([64, 72])


In [60]:
" | ".join(num.vocab[i] for i in X[0][:20])

'xxbos | xxmaj | this | is | one | of | the | best | films | i | have | ever | seen | ! | xxmaj | how | anyone | can | xxunk | this'

In [61]:
" | ".join(num.vocab[i] for i in y[0][:20])

'xxmaj | this | is | one | of | the | best | films | i | have | ever | seen | ! | xxmaj | how | anyone | can | xxunk | this | movie'

### Обучение классификатора текста

с помощью **переноса обучения** (transfer learning) классификатор текста обучается в два этапа
- Тонко настроем (fine-tune) предварительно наученную на корпусе (**википедии**) языковую модель на корпусе (**IMDB**)
- Только после этого эту модель можно использовать для классификации

#### Создание языковой модели с помощью DataBlock

In [62]:
from fastai.text.all import partial, DataBlock, TextBlock, RandomSplitter

get_imdb = partial(get_text_files,folders=['train','test','unsup'])

In [63]:
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path,is_lm=True),
    get_items=get_imdb,
    splitter=RandomSplitter(0.1)
).dataloaders(path,path=path,bs=128,seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj this film is puree of heist movies : the xxmaj comic xxmaj heist ( ocean 's xxmaj eleven ) , the xxmaj sexy xxmaj heist ( the xxmaj thomas xxmaj crown xxmaj affair ) , the xxmaj one xxmaj last xxmaj heist ( the xxmaj score ) , and the "" gee , i xxmaj kinda xxmaj like the xxmaj guy xxmaj i 'm xxmaj chasing / xxmaj being xxmaj chased xxmaj by "" xxmaj heist ( heat","xxmaj this film is puree of heist movies : the xxmaj comic xxmaj heist ( ocean 's xxmaj eleven ) , the xxmaj sexy xxmaj heist ( the xxmaj thomas xxmaj crown xxmaj affair ) , the xxmaj one xxmaj last xxmaj heist ( the xxmaj score ) , and the "" gee , i xxmaj kinda xxmaj like the xxmaj guy xxmaj i 'm xxmaj chasing / xxmaj being xxmaj chased xxmaj by "" xxmaj heist ( heat )"
1,", xxup please xxup see xxup this xxup film ! xxmaj we can all learn to live life , and stop all these small problems . xxmaj because when someone you love almost passes away or does pass away , everything does n't matter anymore . \n\n 10 out of 10 xxbos xxmaj in watching this early demille work , it was once again reinforced to me that early demille is far superior to late demille . xxmaj his attention","xxup please xxup see xxup this xxup film ! xxmaj we can all learn to live life , and stop all these small problems . xxmaj because when someone you love almost passes away or does pass away , everything does n't matter anymore . \n\n 10 out of 10 xxbos xxmaj in watching this early demille work , it was once again reinforced to me that early demille is far superior to late demille . xxmaj his attention to"


#### Тонкая настройка языковой модели

*   List item
*   List item



In [ ]:
from fastai.text.all import language_model_learner, AWD_LSTM, accuracy, Perplexity

learn = language_model_learner(dls_lm,AWD_LSTM,drop_mult=0.3,
                               metrics=[accuracy,Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1,2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.006543,3.899345,0.300549,49.370087,26:29


#### Сохранение и загрузка модели

In [ ]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [ ]:
learn.load('1epoch')